In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

file_path = './dog.txt'
data = pd.read_csv(file_path, sep='\t') 

import pandas as pd

# Pad sequences to the same length
min_length = min(data['sequence'].apply(len))   #we can try with max(it might probably give better results since using min, we're not considering all the repeat patterns in the sequence that might be responsible for making it to be in a particular cluster )
def pad_sequence(sequence, min_length):         #I used min to make the code run faster as max seemed to take hours.
    if len(sequence) >= min_length:
        padded_sequence = sequence[:min_length]
    else:
        padding = "N" * (min_length - len(sequence))
        padded_sequence = sequence + padding
    return padded_sequence

# Pad all sequences in the dataframe to the same length 
data['sequence'] = data['sequence'].apply(lambda x: pad_sequence(x, min_length))

# # Print the padded sequences dataframe
# print(data['sequence'])

import numpy as np

def one_hot_encode(sequence):
    encoder = OneHotEncoder(categories=[['A', 'C', 'G', 'T','N']])
    sequence_array = np.array(list(sequence)).reshape(-1, 1)
    encoded = encoder.fit_transform(sequence_array).toarray()
    return encoded

data['encoded_sequence'] = data['sequence'].apply(lambda x: one_hot_encode(x))

from sklearn.model_selection import train_test_split

# Split the one-hot encoded sequences and the labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data["encoded_sequence"].to_numpy(), data["class"].to_numpy(), test_size=0.3, random_state=42)

# # Check the shapes of the resulting sets
# print("X_train shape:", len(X_train))
# print("y_train shape:", y_train.shape)
# print("X_test shape:", len(X_test))
# print("y_test shape:", y_test.shape)

import numpy as np

# Convert the input data into NumPy arrays with dtype float32
X_train_np = np.array(X_train.tolist(), dtype=np.float32)
y_train_np = np.array(y_train.tolist(), dtype=np.float32)
X_test_np = np.array(X_test.tolist(), dtype=np.float32)
y_test_np = np.array(y_test.tolist(), dtype=np.float32)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Build the model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(None, 5)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(34, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model using padded batch
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train_np)).padded_batch(32, padded_shapes=((None, 5), ()), drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y_test_np)).padded_batch(32, padded_shapes=((None, 5), ()), drop_remainder=True)

history = model.fit(train_dataset, epochs=200, validation_data=val_dataset, verbose=1, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(val_dataset, verbose=1)

print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")



Epoch 1/200
17/17 [==============================] - 11s 293ms/step - loss: 1.8810 - accuracy: 0.2812 - val_loss: 1.8163 - val_accuracy: 0.3304
Epoch 2/200
17/17 [==============================] - 4s 209ms/step - loss: 1.8610 - accuracy: 0.2960 - val_loss: 1.8071 - val_accuracy: 0.3304
Epoch 3/200
17/17 [==============================] - 3s 198ms/step - loss: 1.8290 - accuracy: 0.3143 - val_loss: 1.7968 - val_accuracy: 0.3304
Epoch 4/200
17/17 [==============================] - 3s 193ms/step - loss: 1.8328 - accuracy: 0.2904 - val_loss: 1.7902 - val_accuracy: 0.3304
Epoch 5/200
17/17 [==============================] - 3s 198ms/step - loss: 1.8188 - accuracy: 0.3125 - val_loss: 1.7540 - val_accuracy: 0.3616
Epoch 6/200
17/17 [==============================] - 5s 272ms/step - loss: 1.8208 - accuracy: 0.3272 - val_loss: 1.8065 - val_accuracy: 0.2545
Epoch 7/200
17/17 [==============================] - 4s 259ms/step - loss: 1.8135 - accuracy: 0.3346 - val_loss: 1.7477 - val_accuracy: 0.361